### Overview

Provide some examples of how to use the `attribution` package.

In [1]:
# Load example data
import json

with open('../data/example_data.json', 'r') as f:
    data = json.load(f)

# Inspect subset of data
data[:10]

[{'identifier': 1, 'impressions': ['B', 'A'], 'value': 944},
 {'identifier': 2, 'impressions': ['B', 'A'], 'value': 952},
 {'identifier': 3, 'impressions': ['B', 'A'], 'value': 776},
 {'identifier': 4, 'impressions': ['C', 'B', 'A'], 'value': 1389},
 {'identifier': 5, 'impressions': ['C', 'B'], 'value': 717},
 {'identifier': 6, 'impressions': ['B'], 'value': 628},
 {'identifier': 7, 'impressions': ['A'], 'value': 332},
 {'identifier': 8, 'impressions': ['C', 'A'], 'value': 433},
 {'identifier': 9, 'impressions': ['C'], 'value': 199},
 {'identifier': 10, 'impressions': ['A'], 'value': 68}]

### Attribution Methods



In [2]:
from attribution.valuator import build_valuator

valuator = build_valuator(data, 'first_touch')

valuator.valuate_treatments()

{'A': 875995, 'B': 3503081, 'C': 2741803}

In [3]:
valuator = build_valuator(data, 'last_touch')

valuator.valuate_treatments()

{'A': 6122920, 'B': 915703, 'C': 82256}

In [4]:
valuator = build_valuator(data, 'shapley')

22/05/06 21:02:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/06 21:02:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/05/06 21:02:49 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [5]:
valuator.valuate_treatments()

{'A': 3159066.0, 'B': 2890174.9999999995, 'C': 1071638.0}

In [6]:
3159066.0 + 2890174.9999999995 + 1071638.0

7120879.0

In [7]:
6122920 + 915703 + 82256

7120879

In [6]:
from attribution.audience import build_audience

audience = build_audience(data)

# # audience.to_pyspark()
# a = build_audience([dict(identifier='1', **audience.members[0].journey.to_dict())])
# j = a.members[0].journey

# list(j.impressions)
# j.impressions = list(j.impressions)

# build_audience([dict(identifier='1', **j.to_dict())]).to_pyspark()

DataFrame[identifier: string, treatment_set: array<string>, value: bigint, A: int, B: int]

In [ ]:
from attribution.audience import build_audience
from attribution.journey import build_journey_set

audience = build_audience(data)

treatments = audience.treatments

possible_journeys = build_journey_set(treatments)

# possible_journeys
# Massage into a data frame with identifiers, etc.
#  We'll leverage an "audience" to do this
for i, j in enumerate(possible_journeys):
    if i == 0:
        data = list()

    # Append a sequential identifier
    _data = dict(
        identifier=i,
        **j.to_dict()
    )

data.append(_data)

journey_set = build_audience(data).to_pyspark()

In [ ]:
builder = build_valuator(data, 'shapley')

In [ ]:
audience_data = builder.audience.to_pyspark()

attribution_data = audience_data

treatments = builder.audience.treatments

from attribution.journey import build_journey_set

possible_journeys = build_journey_set(treatments)


In [ ]:
possible_journeys

In [ ]:
# Massage into a data frame with identifiers, etc.
#  We'll leverage an "audience" to do this
for i, j in enumerate(possible_journeys):
    if i == 0:
        data = list()

      # Append a sequential identifier
    _data = dict(identifier=i, **j.to_dict())
    data.append(_data)
    
a = build_audience(data)

row_data = []
for m in a.members:
    row_data.append(m.to_row())

from pyspark.sql import SparkSession
import pyspark.sql.functions as sf

spark = SparkSession.builder.getOrCreate()

spark.createDataFrame(row_data).withColumnRenamed('impressions', 'treatment').select(sf.posexplode('treatment'))